# Types and type declarations

In [1]:
x = 100.0
typeof(x)

Float64

In [2]:
function foo()
    x::Int8 = 100
    x
end

x = foo()
typeof(x)

Int8

## Type trees

In [3]:
using AbstractTrees
AbstractTrees.children(x::Type) = subtypes(x)

LoadError: ArgumentError: Package AbstractTrees not found in current path:
- Run `import Pkg; Pkg.add("AbstractTrees")` to install the AbstractTrees package.


In [ ]:
print_tree(Any)

In [ ]:
print_tree(AbstractArray)

## Types in function arguments

In [ ]:
function g(x, y)
    return x * y
end

x = g(2, 3)
@show x
typeof(x)

In [ ]:
x = g(2.0, 3.0)
@show x
typeof(x)

In [ ]:
x = g("foo", "bar")
@show x
typeof(x)

In [ ]:
function f(x::Number, y::Number)
    return x * y
end

x = f(2, 3)
@show x
typeof(x)

In [ ]:
x = f("foo", "bar")

## Type Unions

In [ ]:
function h(x::Union{Integer, String}, y::Union{Integer, String})
    x * y
end

@show h(1, 2)
@show h("foo", "bar");

In [ ]:
h(1.0, 2.0)

## Composite Types

In [ ]:
struct Point2D
    x::Float64
    y::Float64
end

p = Point2D(1.0, 2.0)
@show p.x
@show p.y;

### Parametric Composite Types

In [ ]:
struct Point3D{T}
    x::T
    y::T
    z::T
end

p = Point3D{Int8}(1, 2, 3)
@show p.x
typeof(p.y)

In [ ]:
p = Point3D{Float32}(1.0, 2.0, 3.0)
typeof(p.x)

In [ ]:
p = Point3D{String}("foo", "bar", "foobar")
typeof(p.x)

### Type restrictions on parameters

In [ ]:
struct NumericPoint2D{T<:Real}
    x::T
    y::T
end 

p = NumericPoint2D{Float64}(1.0, 2.0)
@show typeof(p.x)

q = NumericPoint2D{Int8}(1, 2)
@show typeof(q.x)

In [ ]:
NumericPoint2D{String}("foo", "bar")

In [ ]:
import Base.+

### Functions and operators with parametric types

In [ ]:
+(A::NumericPoint2D{T}, B::NumericPoint2D{T}) where T = NumericPoint2D{T}(A.x + B.x, A.y + B.y)

In [ ]:
A = NumericPoint2D{Int32}(1, 2)
B = NumericPoint2D{Int32}(1, 2)

A + B

**Fields of composite types are immutable by default**

In [ ]:
A.x = 2

## Mutable composite types

In [ ]:
mutable struct MutablePoint2D{T<:Real}
    x::T
    y::T
end 

A = MutablePoint2D{Int32}(1, 2)
A.x = 2
@show A.x;

## Operations on types

In [ ]:
isa(1, Int)

In [ ]:
typeof(1)

In [ ]:
supertype(UInt32)

## Constructors

In [ ]:
struct Polar{T<:Real}
    r::T
    θ::T
end

A = Polar(1.0, π / 4)
A.θ

In [ ]:
function Polar(A::NumericPoint2D{T}) where T
    r = sqrt(A.x ^ 2 + A.y ^ 2)
    θ = atan(A.y, A.x)
    Polar{T}(r, θ)
end

In [ ]:
A = NumericPoint2D{Float32}(cos(π / 4), sin(π / 4))
B = Polar(A)
@show B.r
@show B.θ
typeof(B.r)

In [ ]:
macro javascript_str(s) display("text/javascript", s); end
javascript"""
function hideElements(elements, start) {
for(var i = 0, length = elements.length; i < length;i++) {
    if(i >= start) {
        elements[i].style.display = "none";
    }
}
}
var prompt_elements = document.getElementsByClassName("prompt");
hideElements(prompt_elements, 0)
"""